**Problem Statement**

A large company named XYZ, employs, at any given point of time, around 4000 employees. 
However, every year, around 15% of its employees leave the company and need to be replaced with the talent pool available in the job market. 
The management believes that this level of attrition (employees leaving, either on their own or because they got fired) is bad for the company, because of the following reasons:

1. The former employees’ projects get delayed, which makes it difficult to meet timelines, resulting in a reputation loss among consumers and partners
2. A sizeable department has to be maintained, for the purposes of recruiting new talent
3. More often than not, the new employees have to be trained for the job and/or given time to acclimatise themselves to the company

*The management has contracted an HR analytics firm to curb attrition.* 

-> What changes they should make to their workplace to get most of their employees to stay. 

-> Which of these variables is most important and needs to be addressed right away.


**Goal of the case study**

You are required to model the probability of attrition using a logistic regression. The results thus obtained will be used by the management to understand what changes they should make to their workplace, in order to get most of their employees to stay.

### Import and Merge data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
hr_df = pd.read_csv(r'C:\Users\thuy.nguyen\OneDrive - FiinGroup JSC\Desktop\dataset\HR\general_data.csv')
NPS_df = pd.read_csv(r'C:\Users\thuy.nguyen\OneDrive - FiinGroup JSC\Desktop\dataset\HR\employee_survey_data.csv')
performance_df = pd.read_csv(r'C:\Users\thuy.nguyen\OneDrive - FiinGroup JSC\Desktop\dataset\HR\manager_survey_data.csv')
intime_df = pd.read_csv(r'C:\Users\thuy.nguyen\OneDrive - FiinGroup JSC\Desktop\dataset\HR\in_time.csv')
outtime_df = pd.read_csv(r'C:\Users\thuy.nguyen\OneDrive - FiinGroup JSC\Desktop\dataset\HR\out_time.csv')

In [3]:
intime_df.head()

,EmployeeID,1/1/2015,1/2/2015,1/5/2015,1/6/2015,1/7/2015,1/8/2015,1/9/2015,1/12/2015,1/13/2015,...,12/18/2015,12/21/2015,12/22/2015,12/23/2015,12/24/2015,12/25/2015,12/28/2015,12/29/2015,12/30/2015,12/31/2015
0,1,NaN,9:43:45 AM,10:08:48 AM,9:54:26 AM,9:34:31 AM,9:51:09 AM,10:09:25 AM,9:42:53 AM,10:13:06 AM,...,NaN,9:55:29 AM,10:04:06 AM,10:14:27 AM,10:11:35 AM,NaN,10:13:41 AM,10:03:36 AM,9:54:12 AM,10:12:44 AM
1,2,NaN,10:15:44 AM,10:21:05 AM,NaN,9:45:17 AM,10:09:04 AM,9:43:26 AM,10:00:07 AM,10:43:29 AM,...,10:37:17 AM,9:49:02 AM,10:33:51 AM,10:12:10 AM,NaN,NaN,9:31:45 AM,9:55:49 AM,10:32:25 AM,9:27:20 AM
2,3,NaN,10:17:41 AM,9:50:50 AM,10:14:13 AM,9:47:27 AM,10:03:40 AM,10:05:49 AM,10:03:47 AM,10:21:26 AM,...,10:15:14 AM,10:10:28 AM,9:44:44 AM,10:15:54 AM,10:07:26 AM,NaN,9:42:05 AM,9:43:36 AM,9:34:05 AM,10:28:39 AM
3,4,NaN,10:05:06 AM,9:56:32 AM,10:11:07 AM,9:37:30 AM,10:02:08 AM,10:08:12 AM,10:13:42 AM,9:53:22 AM,...,10:17:38 AM,9:58:21 AM,10:04:25 AM,10:11:46 AM,9:43:15 AM,NaN,9:52:44 AM,9:33:16 AM,10:18:12 AM,10:01:15 AM
4,5,NaN,10:28:17 AM,9:49:58 AM,9:45:28 AM,9:49:37 AM,10:19:44 AM,10:00:50 AM,10:29:27 AM,9:59:32 AM,...,9:58:35 AM,10:03:41 AM,10:10:30 AM,10:13:36 AM,9:44:24 AM,NaN,10:05:15 AM,10:30:53 AM,9:18:21 AM,9:41:09 AM


In [4]:
hr_df.info()
#NumCompaniesWorked, TotalWorkingYears have null result

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      4410 non-null   int64  
 1   Attrition                4410 non-null   object 
 2   BusinessTravel           4410 non-null   object 
 3   Department               4410 non-null   object 
 4   DistanceFromHome         4410 non-null   int64  
 5   Education                4410 non-null   int64  
 6   EducationField           4410 non-null   object 
 7   EmployeeCount            4410 non-null   int64  
 8   EmployeeID               4410 non-null   int64  
 9   Gender                   4410 non-null   object 
 10  JobLevel                 4410 non-null   int64  
 11  JobRole                  4410 non-null   object 
 12  MaritalStatus            4410 non-null   object 
 13  MonthlyIncome            4410 non-null   int64  
 14  NumCompaniesWorked      

#### Merge in_time and out_time

In [5]:
# Tranform NA value -> 0
intime_df=intime_df.replace(np.nan,0)

outtime_df=outtime_df.replace(np.nan,0)

In [6]:
intime_df.iloc[:, 1:] = intime_df.iloc[:, 1:].apply(pd.to_datetime, errors='coerce')
outtime_df.iloc[:, 1:] = outtime_df.iloc[:, 1:].apply(pd.to_datetime, errors='coerce')

C:\Users\thuy.nguyen\AppData\Local\Temp\ipykernel_11996\3840926934.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  intime_df.iloc[:, 1:] = intime_df.iloc[:, 1:].apply(pd.to_datetime, errors='coerce')
C:\Users\thuy.nguyen\AppData\Local\Temp\ipykernel_11996\3840926934.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  outtime_df.iloc[:, 1:] = outtime_df.iloc[:, 1:].apply(pd.to_datetime, errors='coerce')


In [7]:
# intime_df = intime_df.append(outtime_df)
intime_df = pd.concat([intime_df, outtime_df],keys=['intime_df', 'outtime_df'])

In [8]:
intime_df = intime_df.diff(periods=4410)
intime_df = intime_df.iloc[4410:]
intime_df.reset_index(inplace=True)
intime_df.head()

C:\Users\thuy.nguyen\AppData\Local\Temp\ipykernel_11996\3521141509.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  intime_df.reset_index(inplace=True)
C:\Users\thuy.nguyen\AppData\Local\Temp\ipykernel_11996\3521141509.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  intime_df.reset_index(inplace=True)


,level_0,level_1,EmployeeID,1/1/2015,1/2/2015,1/5/2015,1/6/2015,1/7/2015,1/8/2015,1/9/2015,...,12/18/2015,12/21/2015,12/22/2015,12/23/2015,12/24/2015,12/25/2015,12/28/2015,12/29/2015,12/30/2015,12/31/2015
0,outtime_df,0,0.0,0 days,0 days 07:12:30,0 days 07:11:23,0 days 07:24:39,0 days 07:00:24,0 days 07:17:23,0 days 07:29:04,...,0 days 00:00:00,0 days 07:20:21,0 days 07:23:45,0 days 06:30:17,0 days 07:35:47,0 days,0 days 07:46:26,0 days 07:18:54,0 days 07:46:44,0 days 07:04:49
1,outtime_df,1,0.0,0 days,0 days 08:06:33,0 days 07:27:17,0 days 00:00:00,0 days 07:23:49,0 days 07:25:00,0 days 07:09:03,...,0 days 07:54:11,0 days 07:45:14,0 days 07:42:44,0 days 07:26:08,0 days 00:00:00,0 days,0 days 07:36:53,0 days 07:58:57,0 days 07:59:10,0 days 08:13:38
2,outtime_df,2,0.0,0 days,0 days 06:41:33,0 days 07:15:56,0 days 06:24:19,0 days 06:45:54,0 days 07:20:42,0 days 06:51:41,...,0 days 06:47:09,0 days 07:09:49,0 days 06:48:06,0 days 06:43:49,0 days 06:50:59,0 days,0 days 07:01:26,0 days 07:26:20,0 days 07:32:20,0 days 06:47:11
3,outtime_df,3,0.0,0 days,0 days 07:20:18,0 days 07:17:31,0 days 06:56:35,0 days 06:55:10,0 days 06:51:03,0 days 07:11:35,...,0 days 07:37:45,0 days 06:50:48,0 days 07:19:35,0 days 07:24:49,0 days 07:05:06,0 days,0 days 07:26:50,0 days 07:25:00,0 days 07:21:59,0 days 07:07:59
4,outtime_df,4,0.0,0 days,0 days 08:03:20,0 days 07:59:17,0 days 07:40:57,0 days 07:48:22,0 days 07:39:44,0 days 07:43:18,...,0 days 07:54:13,0 days 07:39:54,0 days 07:57:27,0 days 07:47:13,0 days 08:14:58,0 days,0 days 07:39:44,0 days 08:16:07,0 days 07:57:12,0 days 08:01:05


In [9]:
intime_df.drop(columns=['level_0','level_1','EmployeeID'],axis=1,inplace=True)
intime_df.head()

,1/1/2015,1/2/2015,1/5/2015,1/6/2015,1/7/2015,1/8/2015,1/9/2015,1/12/2015,1/13/2015,1/14/2015,...,12/18/2015,12/21/2015,12/22/2015,12/23/2015,12/24/2015,12/25/2015,12/28/2015,12/29/2015,12/30/2015,12/31/2015
0,0 days,0 days 07:12:30,0 days 07:11:23,0 days 07:24:39,0 days 07:00:24,0 days 07:17:23,0 days 07:29:04,0 days 07:15:46,0 days 07:49:52,0 days,...,0 days 00:00:00,0 days 07:20:21,0 days 07:23:45,0 days 06:30:17,0 days 07:35:47,0 days,0 days 07:46:26,0 days 07:18:54,0 days 07:46:44,0 days 07:04:49
1,0 days,0 days 08:06:33,0 days 07:27:17,0 days 00:00:00,0 days 07:23:49,0 days 07:25:00,0 days 07:09:03,0 days 07:36:41,0 days 07:16:44,0 days,...,0 days 07:54:11,0 days 07:45:14,0 days 07:42:44,0 days 07:26:08,0 days 00:00:00,0 days,0 days 07:36:53,0 days 07:58:57,0 days 07:59:10,0 days 08:13:38
2,0 days,0 days 06:41:33,0 days 07:15:56,0 days 06:24:19,0 days 06:45:54,0 days 07:20:42,0 days 06:51:41,0 days 07:25:07,0 days 06:59:59,0 days,...,0 days 06:47:09,0 days 07:09:49,0 days 06:48:06,0 days 06:43:49,0 days 06:50:59,0 days,0 days 07:01:26,0 days 07:26:20,0 days 07:32:20,0 days 06:47:11
3,0 days,0 days 07:20:18,0 days 07:17:31,0 days 06:56:35,0 days 06:55:10,0 days 06:51:03,0 days 07:11:35,0 days 06:59:55,0 days 07:18:23,0 days,...,0 days 07:37:45,0 days 06:50:48,0 days 07:19:35,0 days 07:24:49,0 days 07:05:06,0 days,0 days 07:26:50,0 days 07:25:00,0 days 07:21:59,0 days 07:07:59
4,0 days,0 days 08:03:20,0 days 07:59:17,0 days 07:40:57,0 days 07:48:22,0 days 07:39:44,0 days 07:43:18,0 days 08:21:54,0 days 08:15:26,0 days,...,0 days 07:54:13,0 days 07:39:54,0 days 07:57:27,0 days 07:47:13,0 days 08:14:58,0 days,0 days 07:39:44,0 days 08:16:07,0 days 07:57:12,0 days 08:01:05


In [10]:
# Drop NA column
intime_df.drop(['1/1/2015', '3/5/2015','5/1/2015','7/17/2015','1/14/2015', '1/26/2015', '11/10/2015'
                 ,'9/17/2015','10/2/2015','11/9/2015','11/11/2015','12/25/2015'], axis = 1, inplace = True)

In [11]:
intime_df['Actual Time'] = intime_df.mean(axis=1)
intime_df['Actual Time'].head()

0   0 days 06:52:12.823293172
1   0 days 07:18:57.493975903
2   0 days 06:48:57.891566265
3   0 days 06:47:21.172690763
4   0 days 07:52:39.220883534
Name: Actual Time, dtype: timedelta64[ns]

In [12]:
intime_df['Hours']=intime_df['Actual Time']/np.timedelta64(1, 'h')
intime_df.head()

,1/2/2015,1/5/2015,1/6/2015,1/7/2015,1/8/2015,1/9/2015,1/12/2015,1/13/2015,1/15/2015,1/16/2015,...,12/21/2015,12/22/2015,12/23/2015,12/24/2015,12/28/2015,12/29/2015,12/30/2015,12/31/2015,Actual Time,Hours
0,0 days 07:12:30,0 days 07:11:23,0 days 07:24:39,0 days 07:00:24,0 days 07:17:23,0 days 07:29:04,0 days 07:15:46,0 days 07:49:52,0 days 07:20:49,0 days 07:16:03,...,0 days 07:20:21,0 days 07:23:45,0 days 06:30:17,0 days 07:35:47,0 days 07:46:26,0 days 07:18:54,0 days 07:46:44,0 days 07:04:49,0 days 06:52:12.823293172,6.870229
1,0 days 08:06:33,0 days 07:27:17,0 days 00:00:00,0 days 07:23:49,0 days 07:25:00,0 days 07:09:03,0 days 07:36:41,0 days 07:16:44,0 days 07:36:47,0 days 07:43:39,...,0 days 07:45:14,0 days 07:42:44,0 days 07:26:08,0 days 00:00:00,0 days 07:36:53,0 days 07:58:57,0 days 07:59:10,0 days 08:13:38,0 days 07:18:57.493975903,7.315971
2,0 days 06:41:33,0 days 07:15:56,0 days 06:24:19,0 days 06:45:54,0 days 07:20:42,0 days 06:51:41,0 days 07:25:07,0 days 06:59:59,0 days 07:26:18,0 days 07:12:37,...,0 days 07:09:49,0 days 06:48:06,0 days 06:43:49,0 days 06:50:59,0 days 07:01:26,0 days 07:26:20,0 days 07:32:20,0 days 06:47:11,0 days 06:48:57.891566265,6.816081
3,0 days 07:20:18,0 days 07:17:31,0 days 06:56:35,0 days 06:55:10,0 days 06:51:03,0 days 07:11:35,0 days 06:59:55,0 days 07:18:23,0 days 06:52:36,0 days 06:54:28,...,0 days 06:50:48,0 days 07:19:35,0 days 07:24:49,0 days 07:05:06,0 days 07:26:50,0 days 07:25:00,0 days 07:21:59,0 days 07:07:59,0 days 06:47:21.172690763,6.789215
4,0 days 08:03:20,0 days 07:59:17,0 days 07:40:57,0 days 07:48:22,0 days 07:39:44,0 days 07:43:18,0 days 08:21:54,0 days 08:15:26,0 days 08:15:36,0 days 08:24:13,...,0 days 07:39:54,0 days 07:57:27,0 days 07:47:13,0 days 08:14:58,0 days 07:39:44,0 days 08:16:07,0 days 07:57:12,0 days 08:01:05,0 days 07:52:39.220883534,7.877561


In [13]:
intime_df.reset_index(inplace=True)
intime_df.head()

,index,1/2/2015,1/5/2015,1/6/2015,1/7/2015,1/8/2015,1/9/2015,1/12/2015,1/13/2015,1/15/2015,...,12/21/2015,12/22/2015,12/23/2015,12/24/2015,12/28/2015,12/29/2015,12/30/2015,12/31/2015,Actual Time,Hours
0,0,0 days 07:12:30,0 days 07:11:23,0 days 07:24:39,0 days 07:00:24,0 days 07:17:23,0 days 07:29:04,0 days 07:15:46,0 days 07:49:52,0 days 07:20:49,...,0 days 07:20:21,0 days 07:23:45,0 days 06:30:17,0 days 07:35:47,0 days 07:46:26,0 days 07:18:54,0 days 07:46:44,0 days 07:04:49,0 days 06:52:12.823293172,6.870229
1,1,0 days 08:06:33,0 days 07:27:17,0 days 00:00:00,0 days 07:23:49,0 days 07:25:00,0 days 07:09:03,0 days 07:36:41,0 days 07:16:44,0 days 07:36:47,...,0 days 07:45:14,0 days 07:42:44,0 days 07:26:08,0 days 00:00:00,0 days 07:36:53,0 days 07:58:57,0 days 07:59:10,0 days 08:13:38,0 days 07:18:57.493975903,7.315971
2,2,0 days 06:41:33,0 days 07:15:56,0 days 06:24:19,0 days 06:45:54,0 days 07:20:42,0 days 06:51:41,0 days 07:25:07,0 days 06:59:59,0 days 07:26:18,...,0 days 07:09:49,0 days 06:48:06,0 days 06:43:49,0 days 06:50:59,0 days 07:01:26,0 days 07:26:20,0 days 07:32:20,0 days 06:47:11,0 days 06:48:57.891566265,6.816081
3,3,0 days 07:20:18,0 days 07:17:31,0 days 06:56:35,0 days 06:55:10,0 days 06:51:03,0 days 07:11:35,0 days 06:59:55,0 days 07:18:23,0 days 06:52:36,...,0 days 06:50:48,0 days 07:19:35,0 days 07:24:49,0 days 07:05:06,0 days 07:26:50,0 days 07:25:00,0 days 07:21:59,0 days 07:07:59,0 days 06:47:21.172690763,6.789215
4,4,0 days 08:03:20,0 days 07:59:17,0 days 07:40:57,0 days 07:48:22,0 days 07:39:44,0 days 07:43:18,0 days 08:21:54,0 days 08:15:26,0 days 08:15:36,...,0 days 07:39:54,0 days 07:57:27,0 days 07:47:13,0 days 08:14:58,0 days 07:39:44,0 days 08:16:07,0 days 07:57:12,0 days 08:01:05,0 days 07:52:39.220883534,7.877561


In [14]:
intime_df.drop(intime_df.columns.difference(['index','Hours']), 1, inplace=True)
intime_df

C:\Users\thuy.nguyen\AppData\Local\Temp\ipykernel_11996\297061567.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  intime_df.drop(intime_df.columns.difference(['index','Hours']), 1, inplace=True)


,index,Hours
0,0,6.870229
1,1,7.315971
2,2,6.816081
3,3,6.789215
4,4,7.877561
...,...,...
4405,4405,8.316921
4406,4406,5.897197
4407,4407,7.149526
4408,4408,9.187612


In [15]:
intime_df.rename(columns={'index': 'EmployeeID'},inplace=True)
intime_df.head()

,EmployeeID,Hours
0,0,6.870229
1,1,7.315971
2,2,6.816081
3,3,6.789215
4,4,7.877561


In [16]:
df_1 = pd.merge(NPS_df, hr_df, how='inner', on='EmployeeID')
hrm_df = pd.merge(performance_df, df_1, how='inner', on='EmployeeID')
hrm_df = pd.merge(intime_df, hrm_df, how='inner', on='EmployeeID')
hrm_df.columns

Index(['EmployeeID', 'Hours', 'JobInvolvement', 'PerformanceRating',
       'EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance', 'Age',
       'Attrition', 'BusinessTravel', 'Department', 'DistanceFromHome',
       'Education', 'EducationField', 'EmployeeCount', 'Gender', 'JobLevel',
       'JobRole', 'MaritalStatus', 'MonthlyIncome', 'NumCompaniesWorked',
       'Over18', 'PercentSalaryHike', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'YearsAtCompany',
       'YearsSinceLastPromotion', 'YearsWithCurrManager'],
      dtype='object')

### EDA

#### Fill missing value

In [17]:
#count rows of missing data for each column
missing_number = hrm_df.isnull().sum().sort_values(ascending=False)
#Calculate percentage of null value
missing_percentage = missing_number/len(hrm_df)
missing_info = pd.concat([missing_number,missing_percentage], axis=1, keys=['missing number','missing percentage'])
missing_info.head(10)

,missing number,missing percentage
WorkLifeBalance,37,0.008392
EnvironmentSatisfaction,25,0.005670
JobSatisfaction,20,0.004536
NumCompaniesWorked,19,0.004309
TotalWorkingYears,8,0.001814
EmployeeID,0,0.000000
JobRole,0,0.000000
YearsSinceLastPromotion,0,0.000000
YearsAtCompany,0,0.000000
TrainingTimesLastYear,0,0.000000


In [18]:
hr_missing = hrm_df.loc[:,['Age', 'Department', 'Education','MonthlyIncome', 'TotalWorkingYears', 'NumCompaniesWorked', 'YearsAtCompany']]
hr_missing

,Age,Department,Education,MonthlyIncome,TotalWorkingYears,NumCompaniesWorked,YearsAtCompany
0,51,Sales,2,131160,1.0,1.0,1
1,31,Research & Development,1,41890,6.0,0.0,5
2,32,Research & Development,4,193280,5.0,1.0,5
3,38,Research & Development,5,83210,13.0,3.0,8
4,32,Research & Development,1,23420,9.0,4.0,6
...,...,...,...,...,...,...,...
4404,29,Sales,3,35390,6.0,1.0,6
4405,42,Research & Development,4,60290,10.0,3.0,3
4406,29,Research & Development,4,26790,10.0,2.0,3
4407,25,Research & Development,2,37020,5.0,0.0,4


In [19]:
# Assume the employee got the 1st job after University enrollment (18 years old)
def get_years(row):
    if row['NumCompaniesWorked'] == 0:
        return row['YearsAtCompany']
    else:
        return row['Age'] - 18

hrm_df['TotalWorkingYears'] = hrm_df.apply(get_years, axis=1)

#for result check
hrm_df.iloc[2367]

EmployeeID                                  2368
Hours                                   7.044587
JobInvolvement                                 3
PerformanceRating                              3
EnvironmentSatisfaction                      3.0
JobSatisfaction                              3.0
WorkLifeBalance                              3.0
Age                                           39
Attrition                                     No
BusinessTravel                     Travel_Rarely
Department                                 Sales
DistanceFromHome                               2
Education                                      4
EducationField                     Life Sciences
EmployeeCount                                  1
Gender                                      Male
JobLevel                                       1
JobRole                    Laboratory Technician
MaritalStatus                           Divorced
MonthlyIncome                              72640
NumCompaniesWorked  

In [20]:
null_values = hr_missing['NumCompaniesWorked'].isna().sum()
null_values

19

In [21]:
# Find median of NumCompaniesWorked to fill in missing value
hrm_df['NumCompaniesWorked'].median()

2.0

In [22]:
hrm_df['NumCompaniesWorked'] = hrm_df['NumCompaniesWorked'].fillna(2)
hrm_df['NumCompaniesWorked'].isnull().sum()

0

In [23]:
nps_missing = hrm_df.loc[:,['EmployeeID', 'EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance']]
nps_missing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4409 entries, 0 to 4408
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   EmployeeID               4409 non-null   int64  
 1   EnvironmentSatisfaction  4384 non-null   float64
 2   JobSatisfaction          4389 non-null   float64
 3   WorkLifeBalance          4372 non-null   float64
dtypes: float64(3), int64(1)
memory usage: 172.2 KB


In [24]:
nps_missing[nps_missing['JobSatisfaction'].isnull()]

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance
40,41,3.0,NaN,3.0
124,125,3.0,NaN,3.0
313,314,1.0,NaN,3.0
586,587,3.0,NaN,3.0
859,860,2.0,NaN,2.0
1195,1196,2.0,NaN,2.0
1468,1469,4.0,NaN,3.0
1678,1679,2.0,NaN,3.0
1909,1910,3.0,NaN,3.0
2182,2183,3.0,NaN,3.0


In [25]:
js = nps_missing.groupby(['EnvironmentSatisfaction', 'WorkLifeBalance']).mean()['JobSatisfaction']
js

EnvironmentSatisfaction  WorkLifeBalance
1.0                      1.0                2.491525
                         2.0                2.824742
                         3.0                2.765784
                         4.0                2.606742
2.0                      1.0                2.394737
                         2.0                2.895197
                         3.0                2.664016
                         4.0                2.760000
3.0                      1.0                3.076923
                         2.0                2.675768
                         3.0                2.704294
                         4.0                2.643312
4.0                      1.0                2.520000
                         2.0                2.931741
                         3.0                2.646772
                         4.0                2.930769
Name: JobSatisfaction, dtype: float64

In [26]:
hrm_df['JobSatisfaction'] = hrm_df.groupby(['EnvironmentSatisfaction', 'WorkLifeBalance'])['JobSatisfaction'].apply(lambda x: x.fillna(x.mean().round()))

C:\Users\thuy.nguyen\AppData\Local\Temp\ipykernel_11996\1693990508.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  hrm_df['JobSatisfaction'] = hrm_df.groupby(['EnvironmentSatisfaction', 'WorkLifeBalance'])['JobSatisfaction'].apply(lambda x: x.fillna(x.mean().round()))


In [27]:
#count rows of missing data for each column
missing_number = hrm_df.isnull().sum().sort_values(ascending=False)
#Calculate percentage of null value
missing_percentage = missing_number/len(hrm_df)
missing_info = pd.concat([missing_number,missing_percentage], axis=1, keys=['missing number','missing percentage'])
missing_info.head(10)

,missing number,missing percentage
JobSatisfaction,62,0.014062
WorkLifeBalance,37,0.008392
EnvironmentSatisfaction,25,0.005670
EmployeeID,0,0.000000
JobRole,0,0.000000
YearsSinceLastPromotion,0,0.000000
YearsAtCompany,0,0.000000
TrainingTimesLastYear,0,0.000000
TotalWorkingYears,0,0.000000
StockOptionLevel,0,0.000000


#### Feature Classification: Categorical vs Numerical

In [28]:
hrm_df['Attritrion'] = hrm_df['Attrition'].astype('category')
hrm_df['BusinessTravel'] = hrm_df['BusinessTravel'].astype('category')
hrm_df['Department'] = hrm_df['Department'].astype('category')
hrm_df['Education'] = hrm_df['Education'].astype('category')
hrm_df['EducationField'] = hrm_df['EducationField'].astype('category')
hrm_df['MaritalStatus'] = hrm_df['MaritalStatus'].astype('category')

In [29]:
hrm_df['Education'] = hrm_df['Education'].replace({ 1 : 'Below College', 2: 'College',3: 'Bachelor',4: 'Master', 5: 'Doctor'})
hrm_df['EnvironmentSatisfaction'] = hrm_df['EnvironmentSatisfaction'].replace({ 1 : 'Low', 2: 'Medium',3: 'High',4: 'Very High'})
hrm_df['JobInvolvement'] = hrm_df['JobInvolvement'].replace({ 1 : 'Low', 2: 'Medium',3: 'High',4: 'Very High'})
hrm_df['JobSatisfaction'] = hrm_df['JobSatisfaction'].replace({ 1 : 'Low', 2: 'Medium',3: 'High',4: 'Very High'})
hrm_df['PerformanceRating'] = hrm_df['PerformanceRating'].replace({ 1 : 'Low', 2: 'Good',3: 'Excellent',4: 'Outstanding'})
hrm_df['WorkLifeBalance'] = hrm_df['WorkLifeBalance'].replace({ 1 : 'Bad', 2: 'Good',3: 'Better',4: 'Best'})

#### Exploratory

- What common reasons of attrition?
    + What types of job engagement (NPS) speak?
    + Demography (gender, marital, distance)

- How difference among job level ~ attrition?
- How job performance affect attrition?
- Is there correlation : (salary, %salary hike) vs (experiment, education level, year at company)?

- Universal reasons of high turnover ate in each department? -> make assumption to apply this comp

    - Sales:
        + Commission-Based Compensation ~ % salary hike
        + Lack of Work-Life Balance ~ NPS
        + Inadequate Training and Support ~ training
    
    - HR:
        + Organizational Restructuring or Outsourcing
        + Limited Influence on Decision-Making
        + Inadequate Resources and Support
    
    - R&D:
        + Market Demand for New Skills
        + Project-Based Work
        + Lack of Resources and Support
        + Collaboration and Team Dynamics